In [3]:
pip install faiss-cpu

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.8 MB 322.4 kB/s eta 0:00:42
   - -------------------------------------- 0.5/13.8 MB 322.4 kB/s eta 0:00:42
   - -------------------------------------- 0.5/13.8 MB 322.4 kB/s eta 0:00:42
   - -------------------------------------- 0.5/13.8 MB 322.4 kB/s eta 0:00:42
   -- ------------------------------------- 0.8/13.8 MB 291.8 kB/s eta 0:00:45
   -- ------------------------------------- 0

In [1]:
# import os
# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'GenAI.settings')  # Correct project name
# import django
# django.setup()
from products.models import Product as ProductModel, UserProductClick

prods = ProductModel.objects.all()
uclics = UserProductClick.objects.all()
for u in uclics:
    print(u)

ModuleNotFoundError: No module named 'products'

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Create rich product descriptions
def create_product_context(row):
    return f"""
    Product: {row['name']}
    Price Range: ${row['min_price']} - ${row['max_price']}
    Categories: {', '.join(row['categories'])}
    Tags: {', '.join(row['tags'])}
    Description: {row['description']}
    """

# Add combined_text column to DataFrame
df['combined_text'] = df.apply(create_product_context, axis=1)

# Generate embeddings and create vector database
embeddings = OpenAIEmbeddings()
product_texts = df['combined_text'].tolist()
vectorstore = FAISS.from_texts(product_texts, embeddings)

In [ ]:
def get_most_common_values(series, top_n=3):
    """
    Get most common values from a Series containing lists
    Example: For categories column with [['Electronics'], ['Electronics', 'Gadgets']]
    """
    # Flatten the lists and count occurrences
    all_values = [item for sublist in series for item in sublist]
    value_counts = pd.Series(all_values).value_counts()
    
    return value_counts.head(top_n).index.tolist()

def get_price_range(df):
    """
    Calculate the preferred price range based on liked products
    """
    # Calculate average of min and max prices
    avg_min_price = df['min_price'].mean()
    avg_max_price = df['max_price'].mean()
    
    # Add some buffer (e.g., ±20%) to create a range
    buffer = 0.2
    price_range = {
        'min': avg_min_price * (1 - buffer),
        'max': avg_max_price * (1 + buffer)
    }
    
    return price_range


In [ ]:
def analyze_user_preferences(liked_products_df):
    # Analyze patterns in user's liked products
    common_categories = get_most_common_values(liked_products_df['categories'])
    price_range = get_price_range(liked_products_df)
    preferred_tags = get_most_common_values(liked_products_df['tags'])
    
    return {
        'categories': common_categories,
        'price_range': price_range,
        'tags': preferred_tags
    }

In [ ]:
# Example usage
user_preferences = analyze_user_preferences(liked_products_df)
"""
Output might look like:
{
    'categories': ['Electronics', 'Gadgets', 'Home'],
    'price_range': {'min': 80.0, 'max': 120.0},
    'tags': ['wireless', 'smart-home', 'bluetooth']
}
"""

In [ ]:
from google.generativeai import GenerativeModel
import google.generativeai as genai

def generate_recommendations(user_profile, all_products_df, n_recommendations=5):
    # Configure Gemini API
    genai.configure(api_key='YOUR_GEMINI_API_KEY')
    model = GenerativeModel('gemini-pro')

    # Create prompt
    prompt = f"""
    Given a user who likes products with these characteristics:
    Categories: {user_profile['categories']}
    Price Range: ${user_profile['price_range']['min']} - ${user_profile['price_range']['max']}
    Tags: {user_profile['tags']}
    
    Based on the following product catalog:
    {all_products_df[['name', 'description', 'categories', 'tags', 'min_price', 'max_price']].to_string()}
    
    Recommend {n_recommendations} specific products from the catalog and explain why each would appeal to this user.
    Format each recommendation as:
    Product Name: [name]
    Reason: [detailed explanation]
    """

    # Generate recommendations
    response = model.generate_content(prompt)
    recommendations = response.text
    
    return recommendations

# Optional if you want you can try

In [ ]:
# # Using Gemini embeddings
# from google.generativeai import embed_content

# # Or keep using OpenAI embeddings
# from langchain.embeddings import OpenAIEmbeddings
# import google.generativeai as genai

# # Configure Gemini
# genai.configure(api_key='YOUR_GEMINI_API_KEY')

# def generate_embeddings(texts):
#     embeddings = []
#     for text in texts:
#         embedding = genai.embed_content(
#             model='models/embedding-001',
#             content=text,
#             task_type='retrieval_document'
#         )
#         embeddings.append(embedding['embedding'])
#     return embeddings

# # Create embeddings
# df['embeddings'] = generate_embeddings(df['combined_text'].tolist())